In [58]:
import pandas as pd
import datetime
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import statsmodels.formula.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

In [2]:
df_X=pd.read_csv("delta_price_symbol_X.csv",index_col=0)
df_A_D =pd.read_csv("delta_price_symbols_A_to_D.csv",index_col=0)
df_E_H =pd.read_csv("delta_price_symbols_E_to_H.csv",index_col=0)

In [22]:
#checking if there is any difference in the indexes
set(df_E_H.index)-set(df_X.index)


set()

In [11]:
#setting the index as datetime for all data
df_X['Date'] = pd.to_datetime(df_X['Date'])
df_X.set_index('Date', inplace=True)

df_A_D['Date'] = pd.to_datetime(df_A_D['Date'])
df_A_D.set_index('Date', inplace=True)

df_E_H['Date'] = pd.to_datetime(df_E_H['Date'])
df_E_H.set_index('Date', inplace=True)


In [23]:
#merging the first two dataframes
df = pd.merge_asof(df_X, df_A_D, on='Date')

In [27]:
#merging the last dataframe

df=pd.merge_asof(df, df_E_H, on='Date')

In [33]:
#setting date as index
df.set_index('Date', inplace=True)

Index(['delta_price_X', 'delta_price_A', 'delta_price_B', 'delta_price_C',
       'delta_price_D', 'delta_price_E', 'delta_price_F', 'delta_price_G',
       'delta_price_H'],
      dtype='object')

In [46]:
#getting formula for ols
def formula_from_cols(df, y):
    return y + ' ~ ' + ' + '.join([col for col in df.columns if not col==y])

In [49]:
formula=formula_from_cols(df,'delta_price_X')

In [51]:
#fitting an ols regression
result = sm.ols(formula=formula, data=df).fit()

In [53]:
#summary of the results
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          delta_price_X   R-squared:                       0.249
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     4.799
Date:                Sat, 01 May 2021   Prob (F-statistic):           4.07e-05
Time:                        17:30:39   Log-Likelihood:                 79.066
No. Observations:                 125   AIC:                            -140.1
Df Residuals:                     116   BIC:                            -114.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0049      0.012      0.410      0.682      -0.019       0.029
delta_price_A     8.2711      5.993      1.380      0.170      -3.600      20.142
delta_price_B    -0.2356      7.329     -0.032      0.974     -14.752      14.281
delta_price_C     9.2752      6.302      1.472      0.144      -3.206      21.757
delta_price_D    -9.1387      7.365     -1.241      0.217     -23.725       5.448
delta_price_E     2.4559      3.349      0.733      0.465      -4.178       9.090
delta_price_F    -2.0187      5.081     -0.397      0.692     -12.082       8.045
delta_price_G   -17.4871      4.205     -4.159      0.000     -25.815      -9.159
delta_price_H    13.7601      5.077      2.710      0.008       3.705      23.816
==============================================================================
Omnibus:                       29.438   Durbin-Watson:                   2.704
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.707
Skew:                           0.920   Prob(JB):                     1.20e-15
Kurtosis:                       6.131   Cond. No.                         969.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## c)The most important variable is the delta_price_G since it has a p value of less than 0.05 at 95 % coinfidence to show that this variable is statistically significant in predicting the variability in the dependent variable.

## d)Based on the results I would remove variables which don't have a statistically significant linear relationship with the dependent variable from the model. Some of them are :-
delta_price_A,

delta_price_B,

delta_price_C,

delta_price_D,

delta_price_E,

delta_price_F,

My final formula will be 
formula1='delta_price_X ~ delta_price_G + delta_price_H'

In [56]:
#fitting an ols regression
result1 = sm.ols(formula='delta_price_X ~ delta_price_G + delta_price_H', data=df).fit()

In [57]:
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          delta_price_X   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     3.030
Date:                Sat, 01 May 2021   Prob (F-statistic):             0.0520
Time:                        17:38:59   Log-Likelihood:                 64.226
No. Observations:                 125   AIC:                            -122.5
Df Residuals:                     122   BIC:                            -114.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0055      0.013      0.422      0.674      -0.020       0.031
delta_price_G    -4.3445      1.765     -2.461      0.015      -7.839      -0.850
delta_price_H     3.1062      1.592      1.952      0.053      -0.045       6.257
==============================================================================
Omnibus:                       44.447   Durbin-Watson:                   2.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              127.253
Skew:                           1.325   Prob(JB):                     2.33e-28
Kurtosis:                       7.172   Cond. No.                         171.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [73]:
# VIF dataframe
X=df.drop(['delta_price_X'],axis=1)
vif_data = pd.DataFrame()
vif_data["feature"] = X.iloc[:].columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

In [78]:
#to check multicollienarity
vif_data.sort_values(by='VIF',ascending=False)

,feature,VIF
7,delta_price_H,31.038473
3,delta_price_D,27.714634
2,delta_price_C,18.473310
0,delta_price_A,17.636775
6,delta_price_G,17.317676
5,delta_price_F,16.075372
1,delta_price_B,15.690421
4,delta_price_E,13.405753


In [82]:
vif_data.sort_values(by='VIF',ascending=False).feature

7    delta_price_H
3    delta_price_D
2    delta_price_C
0    delta_price_A
6    delta_price_G
5    delta_price_F
1    delta_price_B
4    delta_price_E
Name: feature, dtype: object

## Extension of d
However since the R squared decreases drastically I would modify the formula based on the multicollinearity to remove the correlated variables and readjust the formula. For example we can see that delta_price_H & delta_price_D are highly correlated we remove one of them ie. delta_price_D since delta_price_H is highly influential due to the low p value with the dependent variable.

We use the same process to remove one out of the next delta_price_C,delta_price_A,delta_price_F,delta_price_B
delta_price_E and re run the models iteratively till we observe an R squared closer to the one we got initially